In [2]:
import numpy as np
from transformers import VideoMAEImageProcessor, VideoMAEModel

2023-01-11 09:23:58.293839: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-11 09:24:00.075009: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-01-11 09:24:00.075118: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-01-11 09:24:00.075128: W tensorfl

In [3]:
from transformers import VideoMAEImageProcessor, VideoMAEForVideoClassification

In [4]:
model_ckpt = "MCG-NJU/videomae-base"
batch_size = 8

In [5]:
from transformers import VideoMAEFeatureExtractor

In [6]:
#dataset_root_path = '/data/ZHO/formats/ucf101_letters_only_avi/'
dataset_root_path = '/projects/ZHO/formats/ucf101_letters_only/'

feature_extractor = VideoMAEFeatureExtractor.from_pretrained(model_ckpt)
 

/usr/local/lib/python3.8/dist-packages/transformers/models/videomae/feature_extraction_videomae.py:28: FutureWarning: The class VideoMAEFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use VideoMAEImageProcessor instead.
  warnings.warn(


In [7]:
import pathlib

dataset_root_path = pathlib.Path(dataset_root_path)

In [8]:
video_count_train = len(list(dataset_root_path.glob("train/*/*.mp4")))
video_count_val = len(list(dataset_root_path.glob("val/*/*.mp4")))
video_count_test = len(list(dataset_root_path.glob("test/*/*.mp4")))
video_total = video_count_train + video_count_val + video_count_test
print(video_count_train, video_count_val, video_count_test)
print(f"Total videos: {video_total}")

64 32 32
Total videos: 128


In [8]:
all_video_file_paths = (
    list(dataset_root_path.glob("train/*/*.mp4"))
    + list(dataset_root_path.glob("val/*/*.mp4"))
    + list(dataset_root_path.glob("test/*/*.mp4"))
)
all_video_file_paths[:5]

[PosixPath('/projects/ZHO/formats/ucf101_letters_only/train/seen/Ganzo_seen.mp4'),
 PosixPath('/projects/ZHO/formats/ucf101_letters_only/train/seen/Original_seen.mp4'),
 PosixPath('/projects/ZHO/formats/ucf101_letters_only/train/noon/Ganzo_noon.mp4'),
 PosixPath('/projects/ZHO/formats/ucf101_letters_only/train/noon/Original_noon.mp4'),
 PosixPath('/projects/ZHO/formats/ucf101_letters_only/train/sheen/Original_sheen.mp4')]

In [9]:
class_labels = sorted({str(path).split("/")[6] for path in all_video_file_paths})
label2id = {label: i for i, label in enumerate(class_labels)}
id2label = {i: label for label, i in label2id.items()}

print(f"Unique classes: {list(label2id.keys())}.")
print(len(id2label))

Unique classes: ['aeen', 'alif', 'baa', 'daad', 'daal', 'faa', 'gaen', 'haa', 'haa_1', 'hamza_wow', 'hamza_yaa', 'jeem', 'kaaf', 'khaa', 'laa', 'laam', 'meem', 'noon', 'qaaf', 'raa', 'saa', 'saad', 'seen', 'sheen', 'taa', 'taa_1', 'tua', 'wow', 'yaa', 'zaai', 'zaal', 'zua'].
32


In [10]:
import pytorchvideo.data

from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    RemoveKey,
    ShortSideScale,
    UniformTemporalSubsample,
)

from torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    RandomHorizontalFlip,
    Resize,
)

In [11]:
import os

mean = feature_extractor.image_mean
std = feature_extractor.image_std
resize_to = feature_extractor.size['shortest_edge']
print(resize_to)
num_frames_to_sample = 16#model.config.num_frames
sample_rate = 4
fps = 30
#clip_duration = num_frames_to_sample * sample_rate / fps
clip_duration =10
print(clip_duration)

# Training dataset transformations.
train_transform = Compose(
    [
        ApplyTransformToKey(
            key="video",
            transform=Compose(
                [
                    UniformTemporalSubsample(num_frames_to_sample),
                    Lambda(lambda x: x / 255.0),
                    Normalize(mean, std),
                    #RandomShortSideScale(min_size=256, max_size=320),
                    Resize((resize_to,resize_to)),
                    #RandomHorizontalFlip(p=0.5),
                ]
            ),
        ),
    ]
)
# Training dataset.
train_dataset = pytorchvideo.data.labeled_video_dataset(
    data_path=os.path.join(dataset_root_path, "train"),
    #data_path = os.path.join(dataset_root_path,'annotations','trainlist01.txt'),
    clip_sampler=pytorchvideo.data.make_clip_sampler("random", clip_duration),
    decode_audio=False,
    transform=train_transform,
)

# Validation and evaluation datasets' transformations.
val_transform = Compose(
    [
        ApplyTransformToKey(
            key="video",
            transform=Compose(
                [
                    UniformTemporalSubsample(num_frames_to_sample),
                    Lambda(lambda x: x / 255.0),
                    Normalize(mean, std),
                    Resize((resize_to, resize_to)),
                ]
            ),
        ),
    ]

)


test_dataset = pytorchvideo.data.labeled_video_dataset(
    data_path=os.path.join(dataset_root_path, "test"),
    #data_path = os.path.join(dataset_root_path,'annotations','trainlist01.txt'),
    clip_sampler=pytorchvideo.data.make_clip_sampler("uniform", clip_duration),
    decode_audio=False,
    transform=val_transform,
)




224
10


In [12]:
#sample_video = next(iter(train_dataset))
#sample_video.keys()

In [13]:
def investigate_video(sample_video):
    """Utility to investigate the keys present in a single video sample."""
    for k in sample_video:
        if k == "video":
            print(k, sample_video["video"].shape)
        else:
            print(k, sample_video[k])

    print(f"Video label: {id2label[sample_video[k]]}")


#investigate_video(sample_video)

In [14]:
import imageio
import numpy as np
from IPython.display import Image


def unnormalize_img(img):
    """Un-normalizes the image pixels."""
    img = (img * std) + mean
    img = (img * 255).astype("uint8")
    return img.clip(0, 255)


def create_gif(video_tensor, filename="sample.gif"):
    """Prepares a GIF from a video tensor.
    
    The video tensor is expected to have the following shape:
    (num_frames, num_channels, height, width).
    """
    frames = []
    for video_frame in video_tensor:
        frame_unnormalized = unnormalize_img(video_frame.permute(1, 2, 0).numpy())
        frames.append(frame_unnormalized)
    kargs = {"duration": 0.25}
    imageio.mimsave(filename, frames, "GIF", **kargs)
    return filename


def display_gif(video_tensor, gif_name="sample.gif"):
    """Prepares and displays a GIF from a video tensor."""
    video_tensor = video_tensor.permute(1, 0, 2, 3)
    gif_filename = create_gif(video_tensor, gif_name)
    return Image(filename=gif_filename)

In [15]:
#sample_video = next(iter(train_dataset))
#video_tensor = sample_video["video"]
#display_gif(video_tensor)

In [16]:
from torch.utils.data import Dataset
import torch

In [17]:
#### Save processed videos and tensor

from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset,batch_size=32,shuffle = False, num_workers=32)  
test_dataloader = DataLoader(test_dataset,batch_size=32,num_workers=32)  

big_tensor_video_train = np.zeros((64,3,16,224,224))
#big_tensor_label_train = {}
big_tensor_video_train_list = {k:[] for k in range(32)}

big_tensor_video_test = np.zeros((32,3,16,224,224))
#big_tensor_label_test = []


for i,j in enumerate(train_dataloader):

#    input = j['video'].permute(0,2,1,3,4)

    input = j['video'].numpy()
    labels = j['label'].numpy()

    for k in range(input.shape[0]):

        big_tensor_video_train_list[labels[k]].append(input[k])
    
    #big_tensor_video_train.append(input)    
    print(labels)

for k in range(len(big_tensor_video_train_list)):
    for jj in range(2):
        big_tensor_video_train[2*k+jj]=big_tensor_video_train_list[k][jj]

for i,j in enumerate(test_dataloader):

#    input = j['video'].permute(0,2,1,3,4)

    input = j['video'].numpy()
    labels = j['label'].numpy()

    for k in range(input.shape[0]):

        big_tensor_video_test[labels[k]] = input[k]
    print(labels)

[19 28]
[14 10]
[ 7 11]
[21  2]
[ 0 31]
[15 24]
[31 22]
[13  3]
[11  0]
[4 7]
[14  4]
[17 29]
[12 25]
[10 29]
[6 9]
[21 23]
[20 20]
[27  8]
[24 16]
[22 16]
[5 2]
[27  3]
[30 23]
[30  6]
[ 1 15]
[19  5]
[ 8 26]
[18 25]
[28 13]
[ 9 12]
[18  1]
[17 26]
[24]
[31]
[17]
[2]
[3]
[9]
[23]
[7]
[20]
[10]
[6]
[4]
[22]
[26]
[11]
[5]
[18]
[1]
[14]
[21]
[25]
[13]
[19]
[12]
[27]
[15]
[16]
[29]
[8]
[0]
[30]
[28]


In [18]:
big_tensor_video_train_reshaped = np.transpose(big_tensor_video_train, (0,2,1,3,4))
big_tensor_video_test_reshaped  = np.transpose(big_tensor_video_test, (0,2,1,3,4))

np.save('train_video_letters_only_data.npy',big_tensor_video_train_reshaped)
np.save('test_video_letters_only_data.npy',big_tensor_video_test_reshaped)



#np.save('train_video_letters_only_data.npy',big_tensor_video_train)

#np.save('test_video_letters_only_data.npy',big_tensor_video_test)

In [34]:
big_tensor_video_train = np.load('train_video_letters_only_data.npy')
big_tensor_video_test = np.load('test_video_letters_only_data.npy')


In [35]:
big_tensor_video_train.shape

(64, 16, 3, 224, 224)

In [36]:
big_tensor_video_test_1 = np.concatenate([big_tensor_video_train[a:a+1] for a in range(0,len(big_tensor_video_train),2)],axis=0)

print(big_tensor_video_test_1.shape)
big_tensor_video_train_1 = []

for i in range(int(len(big_tensor_video_train)/2)):
    big_tensor_video_train_1.append(big_tensor_video_train[i*2:i*2+1])
    big_tensor_video_train_1.append(big_tensor_video_test[i:i+1])
    
big_tensor_video_train_1 = np.concatenate(big_tensor_video_train_1,axis =0)




big_tensor_video_test_2 = np.concatenate([big_tensor_video_train[a:a+1] for a in range(1,len(big_tensor_video_train),2)],axis =0)

big_tensor_video_train_2 = []

for i in range(int(len(big_tensor_video_train)/2)):
    big_tensor_video_train_2.append(big_tensor_video_train[i*2+1:i*2+2])
    big_tensor_video_train_2.append(big_tensor_video_test[i:i+1])
    
big_tensor_video_train_2 = np.concatenate(big_tensor_video_train_2,axis =0)


big_tensor_video_train_2.shape

np.save('train_video_letters_only_data_1.npy',big_tensor_video_train_1)

np.save('test_video_letters_only_data_1.npy',big_tensor_video_test_1)

np.save('train_video_letters_only_data_2.npy',big_tensor_video_train_2)

np.save('test_video_letters_only_data_2.npy',big_tensor_video_test_2)


(32, 16, 3, 224, 224)


In [38]:
print(big_tensor_video_test_1.shape)
print(big_tensor_video_test_2.shape)
print(big_tensor_video_train_1.shape)
print(big_tensor_video_train_2.shape)

(32, 16, 3, 224, 224)
(32, 16, 3, 224, 224)
(64, 16, 3, 224, 224)
(64, 16, 3, 224, 224)


In [132]:

device='cuda'
class CustomTensorDataset(Dataset):
    """TensorDataset with support of transforms.
    """
    def __init__(self, tensors, transform=None):
        assert all(tensors[0].shape[0] == tensor.shape[0] for tensor in tensors)
        self.tensors = tensors
        self.transform = transform
    def __getitem__(self, index):
        x = self.tensors[index]
        #if self.transform:
        #    x = self.transform(x)
        
        #z = torch.from_numpy(np.array(self.tensors[1][index]))
        #z = self.tensors[1][index] 
        #print(index)
        z = torch.Tensor([int(index/2)])
        return x, z #torch.Tensor(int(index/2))
    def __len__(self):
        return self.tensors.shape[0]



In [133]:
torch.Tensor([int(4.5)])

tensor([4.])

In [166]:

import random
import numpy as np

device='cuda'
class CustomTensorDatasetTrain(CustomTensorDataset):
    """TensorDataset with support of transforms.
    """
    def __init__(self, tensors, n_class =32, transform=None):
        super().__init__(tensors,transform)
        self.class_arange = np.arange(n_class)
        #self.order = []
        self.order= [int(i/2) for i in range(n_class*2)]
        #print(self.order)
        #self.finished_flag = False
    def __getitem__(self, index):
        #x = self.tensors[index]
        #if self.transform:
        #    x = self.transform(x)

        print('index_train:',index)
        #print('index_train:',self.order[index])
        x = self.tensors[self.order[index]]
        z = torch.Tensor([self.order[index]])
        
        return x,z
    def __len__(self):
        return self.tensors.shape[0]
    def shuffle(self):
        np.random.shuffle(self.class_arange)
        self.order = []
        for i in self.class_arange:
            self.order = self.order + [i,i]

In [167]:
order = []
class_arange = np.arange(32)
for i in class_arange:
    order = order + [i,i]
print(order)

[0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24, 25, 25, 26, 26, 27, 27, 28, 28, 29, 29, 30, 30, 31, 31]


In [171]:
train_dataset = CustomTensorDatasetTrain(big_tensor_video_train)
#test_dataset = CustomTensorDataset(big_tensor_video_test)

In [172]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_dataset,batch_size=16,shuffle = False, num_workers=1)  
test_dataloader = DataLoader(test_dataset,batch_size = 16)  

In [173]:
epochs = 3
for epoch in range(epochs):
    print('epoch:',epoch)    
    for x,y in train_dataloader:
        
        print('y:',y)

    train_dataset.shuffle()


epoch: 0
index_train: 0
index_train: 1
index_train: 2
index_train: 3
index_train: 4
index_train: 5
index_train: 6
index_train: 7
index_train: 8
index_train: 9
index_train: 10
index_train: 11
index_train: 12
index_train: 13
index_train: 14
index_train: 15
index_train: 16
index_train: 17
 index_train: 18
index_train: 19
index_train:20
index_train: 21
index_train: 22
index_train: 23
index_train: 24
index_train: 25
index_train: 26
index_train: 27
index_train: 28
index_train: 29
index_train: 30
index_train: 31
y: tensor([[0.],
        [0.],
        [1.],
        [1.],
        [2.],
        [2.],
        [3.],
        [3.],
        [4.],
        [4.],
        [5.],
        [5.],
        [6.],
        [6.],
        [7.],
        [7.]])
index_train: 32
index_train: 33
index_train:index_train: 34
index_train: 35
index_train: 36
index_train: 37
index_train: 38
index_train: 39
index_train: 40
index_train: 41
index_train: 42
index_train: 43
 44
index_train: 45
index_train: 46
index_train: 47
y: te

In [122]:
for j,k in test_dataloader:
    pass
    #print('k:',k)
#for i,j in enumerate(train_dataloader):

In [110]:
#'training':CustomTensorDataset((dataa_x_train_8, dataa_y_train),transform=transform)